In [6]:
import re

from litellm import completion

LLM = "gpt-4o"

# TODO: handle multiple contexts

def split_context_into_units(context: str) -> list[str]:
    # Split the text into lines, preserving headers and sentence structure
    lines = context.splitlines()
    units = []
    
    for line in lines:
        # If it's a header (assuming no period at the end), add as a unit
        if line.strip() and not line.strip().endswith('.'):
            units.append(line.strip())
        else:
            # Split the line into sentences
            sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', line.strip())
            units.extend([s for s in sentences if s])
    return units

def format_context_units_for_prompt(context_units: list[str]) -> str:
    numbered_units = [f"{i + 1}. {unit}" for i, unit in enumerate(context_units)]
    # Join the numbered units back into a string
    return "\n".join(numbered_units)

def add_quotes_to_response(response: str, context_units: list[str]) -> str:
    # Function to replace <quote>X-Y</quote> with the corresponding context units
    def replace_quote(match):
        # Extract the range from the match
        start, end = map(int, match.group(1).split('-'))
        # Get the corresponding context units and join them with a space
        quoted_text = ' '.join(context_units[start:end+1])
        return f"\nQuote: {quoted_text}\n"

    # Use regex to find all occurrences of <quote>X-Y</quote>
    response_with_quotes = re.sub(r'<quote>(\d+-\d+)</quote>', replace_quote, response)
    
    return response_with_quotes

def generate_response(query: str, context: str) -> str:
    context_units = split_context_into_units(context)
    context_for_prompt = format_context_units_for_prompt(context_units)
    prompt = f"""
    Answer the following query using the provided context.
    <query>
    {query}
    </query>
    
    <context>
    {context}
    </context>
    
    Please include a quote from the context to justify your answer.
    However, do not quote the text directly.
    Instead include the numbered range of sentences you want to quote the context using the following format: <quote>START_NUMBER-END_NUMBER</quote>.
    For example, <quote>9-12</quote> would quote sentences 9-12, if you thought those sentences were the right ones to justify your answer.
    Do not quote more than is necessary to provide a justification for your answer.
    """
    messages = [ {"content": prompt, "role": "user"}, ]
    response = completion(model=LLM, messages=messages)["choices"][0]["message"]["content"]
    return add_quotes_to_response(response, context_units)
    

In [7]:
query = "How is the lookback period determined?"

chunk = """
The annuity is not actuarially sound because Sally’s life expectancy 
at purchase was 15.72 years while the guarantee period ends in 20 
years (five-year delay plus 15 years). Example:  Diane purchased an annuity at age 65 with a guarantee 
period of 25 years. The annuity is not actuarially sound because 
Diane’s life expectancy is only 19.50 years. The amount transferred for less than fair market value for an 
annuity that is not actuarially sound is the amount that would be 
paid after the end of the person’s life expectancy. The amount 
transferred for less than fair market value is the value of the 
payments due in the last 5.5 years of the annuity (25 minus 19.50 = 
5.50). Example:  Sally purchased an annuity at age 70 with a guarantee 
period of 15 years and payments starting five years after purchase. The annuity is not actuarially sound because Sally’s life expectancy 
at purchase was 15.72 years while the guarantee period ends in 20 
years. The amount transferred for less than fair market value is the 
value of the payments due in the last 4.28 years of the annuity (20 - 
15.72 = 4.28). The first step in determining the period of time that transfers can be 
evaluated for divestment is determining the baseline date; see 
baseline date in this item. Once the baseline date is established, you determine the look-back 
period. The look back period is 60 months prior to the baseline 
date. Transfers that occur on or after a client’s baseline date must be 
considered for divestment. In addition, transfers that occurred 
within the 60-month look-back period must be considered for 
divestment. LOOK-BACK 
PERIOD 

Entire Period 

BRIDGES ELIGIBILITY MANUAL 

STATE OF MICHIGAN 

DEPARTMENT OF HEALTH & HUMAN SERVICES 

 
 
 

BEM 405: MA DIVESTMENT


BPB 2024-019 

7-1-2024 

Penalty Situation 

Baseline Date 

LESS THAN FAIR 
MARKET VALUE 

A divestment determination is not required unless, sometime during 
the month being tested, the client was in a penalty situation. To be 
in a penalty situation, the client must be eligible for MA (other than 
QDWI) and be one of the following: 

In an LTC facility. • 
•  Approved for MIChoice waiver see BEM 106. •  Approved for the PACE program; see BEM 167. •  Eligible for Home Help. •  Eligible for Home Health. """


generate_response(query, chunk)

'The lookback period is determined by first establishing the baseline date. Once the baseline date is established, the lookback period is defined as the 60 months (5 years) prior to that baseline date. Transfers that occur within this 60-month period, including those on or after the baseline date, must be considered for divestment.\n\nThis answer is based on the context provided in sentences \nQuote: period. The look back period is 60 months prior to the baseline date. Transfers that occur on or after a client’s baseline date must be considered for divestment. In addition, transfers that occurred within the 60-month look-back period must be considered for\n.'